In [377]:
import pandas as pd
from pandas import Series
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="darkgrid")
%matplotlib inline
import warnings
warnings.filterwarnings(action="ignore")
pd.set_option("display.max_columns", None)

In [ ]:
df = pd.read_csv(r"D:\Python\data analysis projects\fifa 21\fifa21 raw data v2.csv")
df.sample(frac=0.1)

In [ ]:
df.info()

In [ ]:
df["Club"].dtype

In [ ]:
df["Club"] = df["Club"].str.strip()
df["Club"]

In [ ]:
df["Contract"].unique()

In [ ]:
# number of players who's free
free_pl = []
# number of players who's On Loan
on_loan_pl = []


for index, row in df.iterrows():
    if "On Loan" in row["Contract"] or "Free" in row["Contract"]:
        print(row["Contract"])
    
    if "On Loan" in row["Contract"]:
        on_loan_pl.append(row["Contract"])
    else:
        free_pl.append(row["Contract"])


In [ ]:
print(f"Numbers of players on loan: {len(on_loan_pl)}\nnumbers of free players: {len(free_pl)}".title())

loan = len(on_loan_pl)
free = len(free_pl)
players = [loan, free]

label = ["On Loan", "Free"]

plt.pie(x=players, labels=label, autopct="%1.1f%%", startangle=90, colors=['lightgray', 'forestgreen'])
plt.axis("equal")

plt.title(label="distribution between free and on loan players".title())
plt.show()

## Most of the players are free players and small percent are on loan to anther clubs

In [385]:
from typing import Any

def extract_contract_info(contract: Series) -> tuple[float | Any, float | Any, int]:
    if "On Loan" in contract or "Free" in contract:
        start_date = np.nan
        end_date = np.nan
        contract_length = 0
    else:
        start_date, end_date = contract.split(" ~ ")
        start_year = int(start_date.strip())
        end_year = int(end_date.strip())
        contract_length = end_year - start_year
    
    return start_date, end_date, contract_length



# create new columns and data and apply the extract function
new_col = ["start date", "end date", "contract length"]
new_data = df["Contract"].apply(func=lambda x: Series(data=extract_contract_info(contract=x)))

for i in range(len(new_col)):
    df.insert(loc=df.columns.get_loc(key="Contract") +1 + i,column=new_col[i], value=new_data[i])

# Position the preferred position for the players

In [ ]:
df

In [387]:
from pandas import DataFrame


def split_column(column: Series):
    column = column.str.strip()
    result_df: DataFrame = column.to_frame()
    options = []

    for idx, value in result_df[result_df.columns[0]].dropna().items():
        for option in value.split(","):

            option = option.strip()

            if option not in result_df.columns:
                options.append(option)
                result_df[option] = False
            
            # Set the option to True for the corresponding row
            result_df.at[idx, option] = True
    return result_df

position = split_column(df["Positions"])


In [ ]:
position.columns

In [ ]:
df.columns

In [ ]:
def categorize_contract(contract: Series):
    if contract == "Free":
        return "Free"
    elif "On Loan" in contract:
        return "On Loan"
    else:
        return "Contact"

# apply the function on the data_frame
df.insert(loc=df.columns.get_loc(key="Contract") + 1, column="contract_status", value=df["Contract"].apply(func=categorize_contract))
df

In [ ]:
position_one = position[['RW', 'ST', 'CF', 'LW', 'GK', 'CAM', 'CM', 'CB', 'CDM',
        'RB', 'RM', 'LM', 'LB', 'RWB', 'LWB']]
position_one.sum().sort_values(ascending=False)

In [ ]:
import plotly.express as px
plt.figure(figsize=(15,6))

# Create horizontal bar plot
sns.barplot(y=position_one.sum().sort_values(ascending=False).index, x=position_one.sum().sort_values(ascending=False), orient="h", color="forestgreen")
plt.xlabel(xlabel="players counts in each position".title())
plt.ylabel(ylabel="players position".title())
plt.title(label="players distribution across the field".title())
plt.tight_layout()
plt.show()


fig = px.bar(
    data_frame=np.round((position_one.sum() * 100 / position_one.count()).sort_values(ascending=False), 2),
    color=(position_one.sum() * 100 / position_one.count()).sort_values(ascending=False).index
)

fig.update_layout(
    width=1500,  # Width in pixels
    height=600,  # Height in pixels
    title="players distribution across the field".title(),
    xaxis_title="players in each position".title(),
    yaxis_title='Percentage',
    xaxis_tickangle=-75  # Rotate x-axis labels for better readability
)
fig.show()


## Height

In [ ]:
df.columns

In [ ]:
df["Height"].dtype

In [ ]:
df["Height"].head(10)

In [396]:
df["Height"] = df["Height"].str.replace("cm", "")

In [397]:
import re

def adjusting_hight_unit(height: Series):
# Pattern to match heights in the format 6'2
    pattern = r"([0-9]+)'([0-9]+)\""

    if match := re.search(pattern=pattern, string=str(height)):
        feet, inches = match.groups()
        # Convert feet to inches
        feet = int(feet) * 12
        inches = int(inches) + feet
        # Convert inches to cm and round to 2 decimal places
        cm = inches * 2.54
        return f"{cm:.2f}"  # Return the result as a string
    else:
        return height

df["Height"] = df["Height"].apply(adjusting_hight_unit)
df["Height"] = df["Height"].astype(np.float64)

## Weights

In [ ]:
df["Weight"].dtype

In [ ]:
df["Weight"].unique()

In [ ]:
df["Weight"].isna().sum()

In [ ]:
def convert_weights(weight):
# Handle missing values or NaN
    if pd.isna(weight):
        return np.nan
    
    # Convert from Kg
    if "kg" in weight.lower():
        weight = weight.replace("kg", "").strip() # Remove 'Kg' and strip any spaces
        return float(weight)
    
    # Convert from lbs to Kg
    elif "lbs" in weight.lower():
        weight = x=weight.replace("lbs", "").strip() # Remove 'lbs' and strip any spaces
        weight = np.round(float(weight) * 0.45359237, 2)
        return weight

    return np.nan

df["Weight"] = df["Weight"].apply(convert_weights)
print(df["Weight"].dtype)
df["Weight"]

In [ ]:
df["Weight"].unique()

In [ ]:
convert_to_kg = ['181lbs', '139lbs', '154lbs', '157lbs', '163lbs', '98kg', '103kg', '99kg', '102kg', '56kg', '101kg', '57kg', '55kg', np.nan, None]
for i in range(len(convert_to_kg)):

    if pd.isna(convert_to_kg[i]):
        convert_to_kg[i] = np.nan
    
    elif "lbs" in convert_to_kg[i]:
        # Extract the number part, convert to float, and convert pounds to kilograms
        pounds = float(convert_to_kg[i].replace("lbs", "").strip())
        kilograms = np.round(pounds * 0.453592, 2)
        convert_to_kg[i] = kilograms
    
    elif "kg" in convert_to_kg[i]:
        convert_to_kg[i] = convert_to_kg[i].replace("kg", "").strip()

print(convert_to_kg)

In [ ]:
import re
email: str = input("enter your mail? ")
print(email)
if re.search(pattern=r".+@.+\.edu", string=email):
    print("valid => Regular expression passed")
else: print("invalid => Regular expression failed")

In [405]:
df.rename(
    columns={"start date": "startDate" , "end date" : "endDate", "contract length" : "contractLengthInYears"},
    inplace=True
)

In [ ]:
plt.pie(df["Preferred Foot"].value_counts().values, labels=df["Preferred Foot"].value_counts().index, autopct="%.2f%%", startangle=90)

In [ ]:
df.columns

In [ ]:
missing_data = df.isna()
for col in missing_data.columns.values.tolist():
    print(missing_data[col].value_counts())
    print("\n")

### Loan Date End
**Will be dropped because it only have approximate of 6 percent of data of total column**

In [409]:
"""Loan Date End
True     17966
False     1013
Name: count, dtype: int64"""

np.round((1013 / (17966 + 1013))*100, 2)
df.drop(columns="Loan Date End", inplace=True )

In [ ]:
print(df["Attacking"].isna().sum())
df["Attacking"].dtype

In [ ]:
print(df["Attacking"].count())
df["Attacking"].unique()


In [412]:
from typing import Literal


def extract_money_unit(money: str) -> None | Literal['K'] | Literal['M'] | Literal['Free Agent']:
    money = money.strip("€").lower()
    if money[-1].isalpha():
        if "k" in money:
            return "K"
        elif "m" in money:
            return "M"
    else:
        return "Free Agent"


def convert_values(market_value: str) -> float:
    market_value = float(re.sub(pattern="[€MK]", repl="", string=market_value))
    return market_value


df.insert(loc=df.columns.get_loc(key="Value") + 1, column="MarketValuation", value=df["Value"].apply(func=extract_money_unit))
df["Value"] = df["Value"].apply(func=convert_values)

df.rename(columns={"Value" : "marketValueInEuro", "↓OVA" : "overall", 'Preferred Foot' : "preferredFoot",
                    'Best Position' : "bestPosition", 'Release Clause' : "releaseClause"},
            inplace=True
)

In [413]:
def extract_wage_unit(money: str) -> Literal['K'] | Literal['euro']:
    money = money.strip("€").lower()
    if money[-1].isalpha():
        if "k" in money:
            return "K"
    return "euro"

df.insert(loc=df.columns.get_loc("Wage") + 1, column="salaryUnit", value=df["Wage"].apply(func=extract_wage_unit))
df["Wage"] = df["Wage"].apply(func=convert_values)
df.rename(columns={"Wage" : "salaryInEuro"}, inplace=True)

In [419]:
df.insert(loc=df.columns.get_loc("releaseClause") + 1, column="releaseClauseUnit", value=df["releaseClause"].apply(func=extract_money_unit))
df["releaseClause"] = df["releaseClause"].apply(func=convert_values)

In [ ]:
no_ending_sign = []

for row in df["releaseClause"]:
    if row[-1].isalpha():
        continue
    else:
        no_ending_sign.append(row)
print(set(no_ending_sign))

In [236]:
un_matched = []
for row in df["playerUrl"]:
    if matches := re.search("^(http:\/\/).+(\/)$", row):
        start, end = matches.groups()
        if start == "http://" and end == "/":
            continue
        else:
            un_matched.append(row)

In [ ]:
un_matched

In [ ]:
pattern = r"^([0-9]+)\-([A-Z-a-z]+)\-([0-9]+)$"

for row in df["photoUrl"]:
    if matches := re.search(pattern=pattern, string=row):
        day,month, year = matches.groups()
        if day == "https://" and month == "cdn" and year == ".png":
            continue
        else:
            un_matched.append(row)

len(un_matched)

In [ ]:
pattern = r"^\d{2}\-\S{3}\-\d{2}$"

for row in df["Joined"]:
    if re.match(pattern=pattern, string=row):
        continue
    else:
        un_matched.append(row)
len(un_matched)